In [ ]:
import vtk
import numpy as np

from io import get_sphere_poly_data



def get_sphere_poly_data():
    resources_dir = Path(__file__).parent / 'resources'
    mesh_path = resources_dir / 'geodesic_polyhedron.vtp'
    if not mesh_path.is_file():
        raise FileNotFoundError(f'{mesh_path} does not exist')
    poly_data = read_poly_data(mesh_path)
    if poly_data.GetNumberOfPoints() == 0:
        message = (
            f'Error reading sphere poly data from {mesh_path}. Contents:'
            f'\n{mesh_path.read_text()}'
        )
        raise FileNotFoundError(message)
    return poly_data


def read_poly_data(path, flip=False):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(str(path))
    reader.Update()
    poly_data = reader.GetOutput()
    if flip:
        from .mesh import flipxy
        poly_data = flipxy(poly_data)
    return poly_data



In [1]:


from myutils import get_sphere_poly_data



sphere_poly_data = get_sphere_poly_data()

print(sphere_poly_data)



vtkPolyData (0xaca890)
  Debug: Off
  Modified Time: 361
  Reference Count: 1
  Registered Events: (none)
  Information: 0x2e18ac0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 364
  Field Data:
    Debug: Off
    Modified Time: 262
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 2562
  Number Of Cells: 5120
  Cell Data:
    Debug: Off
    Modified Time: 270
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x2e19380)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x2e18b10
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 1 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 1 1 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
    Normals: (none)
    TCo

In [2]:
from texture import add_wm_lesion, add_clot, get_texture_image
from mesh import (
    get_resection_poly_data,
    get_ellipsoid_poly_data,
    mesh_to_volume,
)


noisy_poly_data = get_resection_poly_data(
    center_ras,
    radii,
    angles,
    noise_offset=noise_offset,
    sphere_poly_data=sphere_poly_data,
    verbose=verbose,
)
   

wm_lesion_poly_data = scale_poly_data(poly_data, scale_factor, center_ras)

with timer('white matter mesh to volume', verbose):
    wm_lesion_mask = mesh_to_volume(wm_lesion_poly_data, resectable_mask)
    save_debug(wm_lesion_mask)
with timer('white matter blend', verbose):
    image_wm = blend(image, csf_image, wm_lesion_mask, sigmas, pad=pad)
    save_debug(image_wm)
with timer('white matter clean', verbose):
    image_wm_smooth = clean_outside_resectable(
        original_image, image_wm, resectable_mask, gray_matter_mask)
    save_debug(image_wm_smooth)

return image_wm_smooth


ImportError: attempted relative import with no known parent package